PART 1

In [13]:
import pandas as p

'''
    1. mpg:           continuous
    2. cylinders:     multi-valued discrete
    3. displacement:  continuous
    4. horsepower:    continuous
    5. weight:        continuous
    6. acceleration:  continuous
    7. model year:    multi-valued discrete
    8. origin:        multi-valued discrete
    9. car name:      string (unique for each instance)
'''

def discretizer(frame, continousVariables):
    for var in continousVariables:
        frame[var] = p.cut(data[var], bins= [0, data[var].median(), data[var].max()], labels= ['low', 'high'])
    return frame

conVars = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']

data = p.read_csv('auto-mpg.csv')

data = discretizer(data, conVars)
data.astype('category')
data = data.drop('car name', axis=1)
data = data.astype(str)
data

mpg cylinders displacement horsepower weight acceleration model year  \
0    low         8         high       high   high          low         70   
1    low         8         high       high   high          low         70   
2    low         8         high       high   high          low         70   
3    low         8         high       high   high          low         70   
4    low         8         high       high   high          low         70   
..   ...       ...          ...        ...    ...          ...        ...   
94  high         4          low        low    low          low         74   
95  high         4          low        low    low         high         74   
96  high         4          low        low    low         high         74   
97  high         4          low        low    low         high         74   
98  high         4          low        low    low         high         75   

   origin  
0       1  
1       1  
2       1  
3       1  
4       1  
..    ...  
94      2  
95      3  
96      3  
97      2  
98      3  

[99 rows x 8 columns]

PART 2

In [18]:
from pgmpy.estimators import PC
import os
import sys
# estimatedDAG = PC(data.loc[:, data.columns != 'car name'])
#estimatedDAG.estimate().edges()
#https://pgmpy.org/_modules/pgmpy/estimators/PC.html
est = PC(data)

with open(os.devnull, "w") as f:
    sys.stdout = f
    skel, seperating_sets = est.build_skeleton(significance_level=0.01, show_progress= False )
    sys.stdout = sys.__stdout__
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = pdag.to_dag()
print("DAG edges:        ", model.edges())

/home/myfirstbattle/.local/lib/python3.9/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/myfirstbattle/.local/lib/python3.9/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
/home/myfirstbattle/.local/lib/python3.9/site-packages/pgmpy/estimators/CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  f

Undirected edges:  [('cylinders', 'displacement'), ('cylinders', 'acceleration'), ('displacement', 'origin')]
PDAG edges:        [('acceleration', 'cylinders'), ('cylinders', 'acceleration'), ('cylinders', 'displacement'), ('displacement', 'origin'), ('displacement', 'cylinders'), ('origin', 'displacement')]
DAG edges:         [('cylinders', 'acceleration'), ('displacement', 'cylinders'), ('displacement', 'origin')]
{"BDeuScore": "type", "BicScore": "type", "K2Score": "type", "MaximumLikelihoodEstimator": "type", "PC": "type", "TabularCPD": "type", "bestBDSHCModel": "DAG", "bestBICHCmodel": "DAG", "bestK2HCModel": "DAG", "bm": "type", "climb": "HillClimbSearch", "conVars": "list", "data": "DataFrame", "discretizer": "function", "est": "PC", "f": "TextIOWrapper", "hcs": "type", "mle": "MaximumLikelihoodEstimator", "mleModel": "BayesianModel", "model": "DAG", "os": "module", "p": "module", "pdag": "PDAG", "print_full": "function", "seperating_sets": "dict", "skel": "Graph", "sys": "modul

PART 2B

In [8]:
from pgmpy.estimators import HillClimbSearch as hcs
from pgmpy.estimators import BDeuScore, K2Score, BicScore
climb = hcs(data)
bestBICHCmodel = climb.estimate(scoring_method=BicScore(data))
print(bestBICHCmodel.edges())

















  0%|          | 7/1000000 [00:01<78:10:06,  3.55it/s]

[('mpg', 'horsepower'), ('cylinders', 'displacement'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('displacement', 'origin'), ('displacement', 'model year'), ('horsepower', 'acceleration')]


In [9]:

bestBDSHCModel = climb.estimate(scoring_method=BDeuScore(data))
print(bestBDSHCModel.edges())



































  0%|          | 17/1000000 [00:03<52:20:47,  5.31it/s]

[('mpg', 'weight'), ('mpg', 'model year'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('cylinders', 'horsepower'), ('cylinders', 'origin'), ('cylinders', 'acceleration'), ('cylinders', 'model year'), ('displacement', 'cylinders'), ('displacement', 'horsepower'), ('displacement', 'weight'), ('displacement', 'origin'), ('displacement', 'mpg'), ('horsepower', 'mpg'), ('horsepower', 'acceleration'), ('horsepower', 'weight'), ('weight', 'origin')]


In [10]:
bestK2HCModel = climb.estimate(scoring_method=K2Score(data))
print(bestK2HCModel.edges())



























  0%|          | 12/1000000 [00:02<68:17:49,  4.07it/s]

[('mpg', 'horsepower'), ('mpg', 'model year'), ('cylinders', 'displacement'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('cylinders', 'horsepower'), ('cylinders', 'acceleration'), ('cylinders', 'model year'), ('displacement', 'origin'), ('displacement', 'mpg'), ('horsepower', 'acceleration'), ('weight', 'mpg')]


PART 3

3 A

In [11]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel as bm
mleModel = bm(bestBICHCmodel.edges())
mle = MaximumLikelihoodEstimator(mleModel, data)

from pgmpy.factors.discrete.CPD import TabularCPD
#https://stackoverflow.com/questions/70625490/how-to-print-the-printing-full-cpd-from-pgmpy
#credit for printing untruncated tables to console...
def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup

for var in list(data.columns):
    print_full(mle.estimate_cpd(var))

mleModel.local_independencies(list(data.columns))

/home/myfirstbattle/.local/lib/python3.9/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


+-----------+--------------+----------------------+--------------+--------------+
| cylinders | cylinders(3) | cylinders(4)         | cylinders(6) | cylinders(8) |
+-----------+--------------+----------------------+--------------+--------------+
| mpg(high) | 0.0          | 0.9583333333333334   | 0.25         | 0.0          |
+-----------+--------------+----------------------+--------------+--------------+
| mpg(low)  | 1.0          | 0.041666666666666664 | 0.75         | 1.0          |
+-----------+--------------+----------------------+--------------+--------------+
+--------------+----------+
| cylinders(3) | 0.010101 |
+--------------+----------+
| cylinders(4) | 0.484848 |
+--------------+----------+
| cylinders(6) | 0.121212 |
+--------------+----------+
| cylinders(8) | 0.383838 |
+--------------+----------+
+--------------------+--------------+--------------+---------------------+--------------+
| cylinders          | cylinders(3) | cylinders(4) | cylinders(6)        | cylinders

(mpg ⟂ origin, model year, displacement, weight | cylinders)
(displacement ⟂ weight, horsepower, mpg, acceleration | cylinders)
(horsepower ⟂ cylinders, origin, model year, displacement, weight | mpg)
(weight ⟂ horsepower, origin, model year, displacement, acceleration, mpg | cylinders)
(acceleration ⟂ cylinders, origin, model year, displacement, mpg, weight | horsepower)
(model year ⟂ cylinders, horsepower, origin, acceleration, mpg, weight | displacement)
(origin ⟂ cylinders, horsepower, model year, acceleration, mpg, weight | displacement)

3 B